# Artificial Neural Network (Yapay Sinir Ağları)

- İnsan beyininin bilgi işleme şeklini referans alan sınıflandırma ve regresyon problemleri için kullanılabilen kuvvetli makine öğrewnmesi algoritmalarından birisidir.
- **Amaç**, en küçük hata ile tahmin yapabilecek katsayılara erişmektir. 
- Makine öğrenmesi içerisinde görüntü işleme, ses işleme, ve metin işleme gibi problemlerde de kullanılan bir algoritmadır.


- **Çok Katmanlı Yapa Sinir Ağlarının Gerçekleştirilme Basamakları**
- Örnek veri seti toplanır.
- Ağın topolojik yapısına karar verilir.
- Ağda bulunan ağırlıklara başlangıç değeri atanır.
- Örnek veri seti ağda sunulur.
- İleri hesaplama işlemi yapılır.
- Gerçek çıktılar ile tahmin çıktıları karşılaştırılır. Öğrenmenin tamamlanması basamakları gerçekleştirilir.
- **Amaç**, gerçek değerler ile tahmin edilen değerler arasındaki farkı minimum yapmaktır.

**Gerekli Kütüphaneler** 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale, StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import  neighbors
from sklearn.svm import SVR

In [3]:
# Uyarı Mesajları ile Karşılaşmamak için bu kütüphaneyi kullanıyoruz.
from warnings import filterwarnings
filterwarnings("ignore")

# Artificial Neural Network - Model ve Tahmin İşlemleri 

In [4]:
df = pd.read_csv("./Hitters.csv")
# bu csv dosyasının içerisinde eksik gözlemleri(NA) çıkardık.
df = df.dropna()

# Veri seti içerisindeki kategorik değişkenleri dummy değişkenlere çeviriyoruz.
dms = pd.get_dummies(df[["League","Division","NewLeague"]])

# bağımlı değişken
y = df[["Salary"]]

# Veri Setinin içerisinden Bağımlı Değişkeni ve Kategorik değişkenlerin ilk hallerini dışarı bırakıyoruz.
X_ = df.drop(["Salary","League","Division","NewLeague"], axis = 1).astype("float64")

# dms ile X_ birleştirip(concat) bağımsız değişken oluşturduk.
X = pd.concat([X_, dms[["League_N","Division_W","NewLeague_N"]]], axis=1)

# train ve test setlerimizi oluşturuyoruz.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.25,
                                                    random_state= 42) 

**Model**

In [22]:
# StandardScaler, bir dönüştürme görevi vardır. standartlaştırma işlemi yapıldı.
scaler = StandardScaler()

In [6]:
?scaler

Type:        StandardScaler
String form: StandardScaler()
File:        c:\users\halil\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py
Docstring:  
Standardize features by removing the mean and scaling to unit variance

The standard score of a sample `x` is calculated as:

    z = (x - u) / s

where `u` is the mean of the training samples or zero if `with_mean=False`,
and `s` is the standard deviation of the training samples or one if
`with_std=False`.

Centering and scaling happen independently on each feature by computing
the relevant statistics on the samples in the training set. Mean and
standard deviation are then stored to be used on later data using
:meth:`transform`.

Standardization of a dataset is a common requirement for many
machine learning estimators: they might behave badly if the
individual features do not more or less look like standard normally
distributed data (e.g. Gaussian with 0 mean and unit variance).

For instance many elements used in the objective f

In [23]:
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [24]:
# mlp = çok katmanlı algılayıcı
mlp_model = MLPRegressor().fit(X_train_scaled,y_train)

In [25]:
?mlp_model

Type:        MLPRegressor
String form: MLPRegressor()
File:        c:\users\halil\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py
Docstring:  
Multi-layer Perceptron regressor.

This model optimizes the squared-loss using LBFGS or stochastic gradient
descent.

.. versionadded:: 0.18

Parameters
----------
hidden_layer_sizes : tuple, length = n_layers - 2, default=(100,)
    The ith element represents the number of neurons in the ith
    hidden layer.

activation : {'identity', 'logistic', 'tanh', 'relu'}, default='relu'
    Activation function for the hidden layer.

    - 'identity', no-op activation, useful to implement linear bottleneck,
      returns f(x) = x

    - 'logistic', the logistic sigmoid function,
      returns f(x) = 1 / (1 + exp(-x)).

    - 'tanh', the hyperbolic tan function,
      returns f(x) = tanh(x).

    - 'relu', the rectified linear unit function,
      returns f(x) = max(0, x)

solver : {'lbfgs', 'sgd', 'adam'}, default='adam'
   

**Tahmin**

In [26]:
mlp_model.predict(X_test_scaled)[0:5]

array([16.30176257, 28.59628561, 56.25911854,  9.48499887, 27.02212038])

In [27]:
y_pred = mlp_model.predict(X_test_scaled)
np.sqrt(mean_squared_error(y_test,y_pred))

663.0095368634517

# Model Tuning

In [28]:
mlp_params = {"alpha": [0.1, 0.01, 0.02, 0.001, 0.0001],
             "hidden_layer_sizes": [(10,20), (5,5),(100,100)]
             }

In [38]:
mlp_cv_model = GridSearchCV(mlp_model,mlp_params,cv=10, verbose = 2, n_jobs= -1).fit(X_train_scaled, y_train)

Fitting 10 folds for each of 15 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:   12.3s finished


In [39]:
mlp_cv_model.best_params_

{'alpha': 0.02, 'hidden_layer_sizes': (100, 100)}

**Final Modeli**

In [43]:
mlp_tuned = MLPRegressor(alpha=0.02, hidden_layer_sizes = (100, 100)).fit(X_train_scaled,y_train)

In [44]:
y_tuned_pred = mlp_tuned.predict(X_test_scaled)

In [45]:
np.sqrt(mean_squared_error(y_test,y_tuned_pred))

354.3546746419776